In [24]:
import pandas as pd

path = "../data/valeurs_foncieres_2022.csv"

# Infos

In [25]:
df = pd.read_csv(path)
display(df.head(3)) 
df.info()

/var/folders/4n/x1d7p39905dgkcjwg2q8g_9w0000gn/T/ipykernel_57167/4026636684.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
0,2022-1102100,2022-01-04,1,Vente,30000.0,9.0,NaN,RUE DE LA MONNAIE,4825,69002.0,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,4.832773,45.761773
1,2022-1102101,2022-01-06,1,Vente,646800.0,2.0,B,MTE DU CHANGE,1425,69005.0,...,Appartement,98.0,4.0,NaN,NaN,NaN,NaN,NaN,4.826967,45.764678
2,2022-1102101,2022-01-06,1,Vente,646800.0,2.0,B,MTE DU CHANGE,1425,69005.0,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,4.826967,45.764678


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107761 entries, 0 to 107760
Data columns (total 40 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   id_mutation                   107761 non-null  object 
 1   date_mutation                 107761 non-null  object 
 2   numero_disposition            107761 non-null  int64  
 3   nature_mutation               107761 non-null  object 
 4   valeur_fonciere               107491 non-null  float64
 5   adresse_numero                86973 non-null   float64
 6   adresse_suffixe               5689 non-null    object 
 7   adresse_nom_voie              106236 non-null  object 
 8   adresse_code_voie             106236 non-null  object 
 9   code_postal                   106236 non-null  float64
 10  code_commune                  107761 non-null  int64  
 11  nom_commune                   107761 non-null  object 
 12  code_departement              107761 non-nul

Nettoyage général

In [26]:
# Standardiser les noms de colonnes
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Supprimer les colonnes entièrement vides
df = df.dropna(axis=1, how="all")

# Supprimer les doublons
df = df.drop_duplicates()

# Supprimer les colonnes trop vides (> 75% de valeurs manquantes)
missing_ratio = df.isnull().mean()
df = df.loc[:, missing_ratio < 0.75]

# Convertir en minuscules pour harmoniser les textes
for col in df.select_dtypes(include="object"):
    df[col] = df[col].str.lower()

Conversion des types et traitement des codes

In [27]:
# Conversion des dates
for col in df.columns:
    if "date" in col:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# Code commune au format texte (5 chiffres)
if "code_commune" in df.columns:
    df["code_commune"] = df["code_commune"].astype(str).str.zfill(5)

Nettoyage des valeurs aberrantes


In [28]:
# Colonnes numériques principales
num_cols = ["valeur_fonciere", "nombre_lots", "surface_reelle_bati",
            "nombre_pieces_principales", "surface_terrain"]

for col in num_cols:
    if col in df.columns:
        # Remplacer valeurs négatives par 0
        df[col] = df[col].apply(lambda x: max(x, 0) if pd.notnull(x) else x)
        # Remplacer les NaN par la médiane
        df[col] = df[col].fillna(df[col].median())

Agrégation par commune

In [29]:
agg_dvf = (
    df.groupby("code_commune", as_index=False)
    .agg(
        nb_ventes=("valeur_fonciere", "count"),
        prix_median=("valeur_fonciere", "median"),
        prix_moyen=("valeur_fonciere", "mean"),
        surface_moyenne=("surface_reelle_bati", "mean"),
        pieces_moyennes=("nombre_pieces_principales", "mean")
    )
)

# Arrondir pour la lisibilité
agg_dvf[["prix_median", "prix_moyen", "surface_moyenne", "pieces_moyennes"]] = \
    agg_dvf[["prix_median", "prix_moyen", "surface_moyenne", "pieces_moyennes"]].round(0)

print(f"✅ Agrégation terminée : {agg_dvf.shape[0]} communes trouvées")

# Aperçu
display(agg_dvf.head(10))

✅ Agrégation terminée : 275 communes trouvées


,code_commune,nb_ventes,prix_median,prix_moyen,surface_moyenne,pieces_moyennes
0,69001,31,200000.0,214459.0,85.0,1.0
1,69002,62,67226.0,96867.0,74.0,1.0
2,69003,109,384750.0,484517.0,80.0,1.0
3,69004,46,389850.0,359887.0,82.0,2.0
4,69005,80,3624.0,202348.0,75.0,1.0
5,69006,295,169000.0,211920.0,148.0,1.0
6,69007,227,357000.0,546789.0,82.0,1.0
7,69008,68,248000.0,220424.0,70.0,1.0
8,69009,495,335650.0,1811738.0,131.0,1.0
9,69010,385,252900.0,350166.0,78.0,1.0


Export 


In [30]:
agg_dvf.to_csv("../csv_clean/valeurs_foncieres_2022_prix_median.csv", index=False)

